# Creating a DeepLabCut Multi-Animal PyTorch Project

- Copy a project to a new folder
- Increase the iteration in the config file
- set `default_net_type: dekr_w32` in config

## Setup

### Imports

In [1]:
import time
import yaml
from pathlib\
import Path

import albumentations as A
import deeplabcut
from deeplabcut.pose_estimation_pytorch import (
    analyze_videos,
    convert_detections2tracklets,
    train_network,
    inference_network,
)

ModuleNotFoundError: No module named 'deeplabcut'

In [2]:
! which python

/Users/rae/.pyenv/shims/python


### Constants

In [ ]:
root = "/Users/niels/Documents/upamathis/datasets"
project = f"{root}/dev-pytorch-ma-pen-2023-06-14"
config_path = f"{project}/config.yaml"

model_prefix = ""

### Helpers

In [ ]:
def read_yaml(path):
    try :
        with open(path, "r") as stream:
            try:
                return yaml.safe_load(stream)
            except yaml.YAMLError as exc:
                print(exc)
    except :
        raise FileNotFoundError("An eero occured whilereading the file")

## Create the PyTorch Training Dataset Files

In [ ]:
shuffles = deeplabcut.create_training_dataset(
    config_path,
    num_shuffles=2,
    net_type="dekr_w18",
)

shuffles = None
if shuffles is not None:
    for shuffle in shuffles:
        print(80 * "-")
        print(f"split = {shuffle[0]}")
        print(f"train_indices = {list(shuffle[2][0])}")
        print(f"test_indices = {list(shuffle[2][1])}")

Go to ```config.yaml``` and set 

- `batch_size: 1`
- `default_track_method: box`


Go to ```dlc-models/iteration-2/devMay17-trainset95shuffle1/train/pytorch_config.yaml``` and set 

- `batch_size: 1`
- `device: cpu`
- `epochs: 100`

## Cool Machine Learning Stuff

### Image Augmentation

In [ ]:
def get_transform(config_path: dict):
    print(80 * "-")
    print("Creating image augmentation transforms")
    cfg = read_yaml(config_path)
    transform = A.Compose(
        [
            A.Affine(
                scale=(0.75, 1.5),
                rotate=(-30, 30),
                translate_px=(-40, 40),
            ),
            A.RandomBrightnessContrast(p=0.5),
            A.MotionBlur(),
            A.PixelDropout(),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ],
        keypoint_params=A.KeypointParams(
            format='xy',
            remove_invisible=False,
            label_fields=['class_labels']
        )
    )
    print(80 * "-")
    return transform


### Training

In [ ]:
def train(config_path, shuffle=1, transform=None, model_prefix=""):
    """Trains the model and evaluates it on a given dataset"""
    # Training the network
    print("Training started")
    start_time = time.time()
    train_network(
        config_path,
        shuffle=shuffle,
        transform=transform,
        model_prefix=model_prefix,
    )
    delta_time = time.time() - start_time
    print(f"Training ended after {delta_time:.2f} seconds")


# No flip transform here; it's confusing the model, better to keep left right for the pen ends
transform = get_transform(config_path)
train(config_path, shuffle=1, transform=transform, model_prefix=model_prefix)

### Evaluation

In [ ]:
test_transform = A.Compose(
    [A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])],
    keypoint_params=A.KeypointParams(
        format='xy',
        remove_invisible=False,
        # label_fields=['class_labels']
    )
)

inference_network(
    config_path,
    shuffle=1,
    model_prefix=model_prefix,
    load_epoch=-1,
    transform=test_transform,
    plot=False,
    evaluate=True,
)

### Video Analysis

In [ ]:
data_path = f"{project}/videos/multipen.mov"
output_folder = f"{project}/videos-analysis-output"

In [ ]:
analysis_transform = A.Compose(
    [A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])],
)

results = analyze_videos(
    config_path,
    data_path=data_path,
    output_folder=output_folder,
    dataset_index=0,
    shuffle=1,
    snapshot_index=-1,
    model_prefix=model_prefix,
    batch_size=1,
    device="cpu",
    transform=analysis_transform,
    overwrite=True,
)

In [ ]:
print(f"Results for {Path(results[0][0]).name}")
results[0][1].head(20)

In [ ]:
convert_detections2tracklets(
    config_path,
    str(data_path),
    dataset_index=0,
    shuffle=1,
    output_folder=output_folder,
    modelprefix="",
    track_method="box",
)
deeplabcut.stitch_tracklets(
    str(config_path),
    [str(data_path)],
    shuffle=1,
    trainingsetindex=0,
    destfolder=str(output_folder),
    n_tracks=2,
    modelprefix="",
    save_as_csv=True,
    track_method="box",
)

In [ ]:
deeplabcut.create_labeled_video(
    config_path,
    [data_path],
    videotype=".mov",
    shuffle=1,
    trainingsetindex=0,
    destfolder=output_folder,
    modelprefix="",
    track_method="box",
)

## Config Files

### `config.yaml`

```yaml
    # Project definitions (do not edit)
Task: dev-ma
scorer: pen
date: Jun14
multianimalproject: true
identity: false

# Project path (change when moving around)
project_path: .../datasets/dev-pytorch-ma-pen-2023-06-14

# Annotation data set configuration (and individual video cropping parameters)
video_sets:
  .../dev-ma-pen-2023-06-14/videos/multipen.mov:
    crop: 0, 480, 0, 640
individuals:
- individual1
- individual2
uniquebodyparts: []
multianimalbodyparts:
- capTip
- bottom
bodyparts: MULTI!

# Fraction of video to start/stop when extracting frames for labeling/refinement
start: 0
stop: 1
numframes2pick: 25

# Plotting configuration
skeleton: []
skeleton_color: black
pcutoff: 0.6
dotsize: 12
alphavalue: 0.7
colormap: rainbow

# Training,Evaluation and Analysis configuration
TrainingFraction:
- 0.8
iteration: 0
default_net_type: dekr_w18
default_augmenter: multi-animal-imgaug
default_track_method: box
snapshotindex: -1
batch_size: 1

# Cropping Parameters (for analysis and outlier frame detection)
cropping: false
# if cropping is true for analysis, then set the values here:
x1: 0
x2: 640
y1: 277
y2: 624

# Refinement configuration (parameters from annotation dataset configuration also relevant in this stage)
corner2move2:
- 50
- 50
move2corner: true
```

### `train/pytorch_config.yaml`

```yaml
batch_size: 1
cfg_path: 
  .../datasets/dev-pytorch-ma-pen-2023-06-14/config.yaml
criterion:
  locref_huber_loss: true
  loss_weight_locref: 0.02
  type: PoseLoss
data:
  covering: true
  gaussian_noise: 12.75
  hist_eq: true
  motion_blur: true
  normalize_images: true
  rotation: 30
  scale_jitter:
  - 0.5
  - 1.25
  translation: 40
device: cpu
display_iters: 1000
epochs: 100
model:
  backbone:
    type: HRNet
    model_name: hrnet_w18
  heatmap_head:
    type: HeatmapDEKRHead
    channels:
    - 270
    - 64
    - 3
    num_blocks: 1
    dilation_rate: 1
    final_conv_kernel: 1
  locref_head:
    type: OffsetDEKRHead
    channels:
    - 270
    - 30
    - 2
    num_offset_per_kpt: 15
    num_blocks: 1
    dilation_rate: 1
    final_conv_kernel: 1
  pose_model:
    stride: 8
  target_generator:
    type: DEKRGenerator
    num_joints: 2
    pos_dist_thresh: 17
optimizer:
  params:
    lr: 0.01
  type: SGD
pos_dist_thresh: 17
predictor:
  type: DEKRPredictor
  num_animals: 2
save_epochs: 50
scheduler:
  params:
    lr_list:
    - - 0.05
    - - 0.005
    milestones:
    - 10
    - 430
  type: LRListScheduler
seed: 42
solver:
  type: BottomUpSingleAnimalSolver
with_center: true
project_path: .../datasets/dev-pytorch-ma-pen-2023-06-14
pose_cfg_path: 
  .../datasets/dev-pytorch-ma-pen-2023-06-14/dlc-models/iteration-0/dev-maJun14-trainset80shuffle1/train/pose_cfg.yaml
```

### `train/pose_cfg.yaml`

```yaml
all_joints:
- - 0
- - 1
all_joints_names:
- capTip
- bottom
alpha_r: 0.02
apply_prob: 0.5
batch_size: 8
contrast:
  clahe: true
  claheratio: 0.1
  histeq: true
  histeqratio: 0.1
convolution:
  edge: false
  emboss:
    alpha:
    - 0.0
    - 1.0
    strength:
    - 0.5
    - 1.5
  embossratio: 0.1
  sharpen: false
  sharpenratio: 0.3
crop_sampling: hybrid
crop_size:
- 400
- 400
cropratio: 0.4
dataset: training-datasets/iteration-0/UnaugmentedDataSet_dev-maJun14/dev-ma_pen80shuffle1.pickle
dataset_type: multi-animal-imgaug
decay_steps: 30000
display_iters: 500
global_scale: 0.8
init_weights: .../DLCdev/deeplabcut
intermediate_supervision: false
intermediate_supervision_layer: 12
location_refinement: true
locref_huber_loss: true
locref_loss_weight: 0.05
locref_stdev: 7.2801
lr_init: 0.0005
max_input_size: 1500
max_shift: 0.4
metadataset: training-datasets/iteration-0/UnaugmentedDataSet_dev-maJun14/Documentation_data-dev-ma_80shuffle1.pickle
min_input_size: 64
mirror: false
multi_stage: false
multi_step:
- - 0.0001
  - 7500
- - 5.0e-05
  - 12000
- - 1.0e-05
  - 200000
net_type: dekr_w18
num_idchannel: 0
num_joints: 2
num_limbs: 1
optimizer: adam
pafwidth: 20
pairwise_huber_loss: false
pairwise_loss_weight: 0.1
pairwise_predict: false
partaffinityfield_graph:
- - 0
  - 1
partaffinityfield_predict: true
pos_dist_thresh: 17
pre_resize: []
project_path: .../datasets/dev-pytorch-ma-pen-2023-06-14
rotation: 25
rotratio: 0.4
save_iters: 10000
scale_jitter_lo: 0.5
scale_jitter_up: 1.25
weigh_only_present_joints: false
```

### `test/inference_cfg.yaml`

```yaml
boundingboxslack: 0
iou_threshold: 0.6
max_age: 1
method: m1
min_hits: 1
minimalnumberofconnections: 1
pafthreshold: 0.1
pcutoff: 0.1
topktoretain: 2
variant: 0
withid: false
```

### `test/pose_cfg.yaml`

```yaml
all_joints:
- - 0
- - 1
all_joints_names:
- capTip
- bottom
dataset: training-datasets/iteration-0/UnaugmentedDataSet_dev-maJun14/dev-ma_pen80shuffle1.pickle
dataset_type: multi-animal-imgaug
global_scale: 0.8
init_weights: .../DLCdev/deeplabcut
location_refinement: true
locref_smooth: false
locref_stdev: 7.2801
minconfidence: 0.01
multi_stage: false
net_type: dekr_w18
nmsradius: 5.0
num_idchannel: 0
num_joints: 2
num_limbs: 1
pairwise_predict: false
partaffinityfield_graph:
- - 0
  - 1
partaffinityfield_predict: true
scoremap_dir: test
sigma: 1
```